# **Fuel Station Refueling example**

## **Scenario:**
### A fuel station has a limited number of fuel pumps that share a common fuel reservoir. Cars randomly arrive at the fuel station, request one of the fuel pumps and start refueling from that reservoir

### A fuel station control process observes the fuel station's fuel level and calls a tank truck for refueling if the station's level drops below a threshold

**Install Simpy**

In [1]:
!pip install simpy

**Imports**

In [0]:
import itertools
import random
import simpy

**Constants**

In [0]:
RANDOM_SEED = 42
FUEL_STATION_SIZE = 200    # liters
THRESHOLD = 10             # Threshold for calling the tank truck (in %)
FUEL_TANK_SIZE = 50        # liters
FUEL_TANK_LEVEL = [5, 25]  # Min/max levels of fuel tanks (in liters)
REFUELING_SPEED = 2        # liters / second
TANK_TRUCK_TIME = 300      # Seconds it takes the tank truck to arrive
T_INTER = [30, 300]        # Create a car every [min, max] seconds
SIM_TIME = 1000            # Simulation time in seconds

**A car arrives at the fuel station for refueling. It requests one of the fuel station's fuel pumps and tries to get the desired amount of fuel from it. If the stations reservoir is depleted, the car has to wait for the tank truck to arrive.**

In [0]:
def car(name, env, fuel_station, fuel_pump):

    fuel_tank_level = random.randint(*FUEL_TANK_LEVEL)
    print('%s arriving at fuel station at %.1f' % (name, env.now))
    
    with fuel_station.request() as req:
        start = env.now
        # Request one of the fuel pumps
        yield req

        # Get the required amount of fuel
        liters_required = FUEL_TANK_SIZE - fuel_tank_level
        yield fuel_pump.get(liters_required)

        # The "actual" refueling process takes some time
        yield env.timeout(liters_required / REFUELING_SPEED)

        print('%s finished refueling in %.1f seconds.' % (name,
                                                          env.now - start))

**Periodically check the level of the *fuel_pump* and call the tank truck if the level falls below a threshold.**

In [0]:
def fuel_station_control(env, fuel_pump):
    
    while True:
        if fuel_pump.level / fuel_pump.capacity * 100 < THRESHOLD:
          
            # We need to call the tank truck now!
            print('Calling tank truck at %d' % env.now)
            
            # Wait for the tank truck to arrive and refuel the station
            yield env.process(tank_truck(env, fuel_pump))

        yield env.timeout(10)  # Check every 10 seconds

**Arrives at the fuel station after a certain delay and refuels it.**

In [0]:
def tank_truck(env, fuel_pump):
    
    yield env.timeout(TANK_TRUCK_TIME)
    print('Tank truck arriving at time %d' % env.now)
    
    ammount = fuel_pump.capacity - fuel_pump.level
    print('Tank truck refuelling %.1f liters.' % ammount)
    
    yield fuel_pump.put(ammount)

**Generate new cars that arrive at the fuel station.**

In [0]:
def car_generator(env, fuel_station, fuel_pump):

    for i in itertools.count():
        yield env.timeout(random.randint(*T_INTER))
        env.process(car('Car %d' % i, env, fuel_station, fuel_pump))

**Execute Simulation**

In [8]:
# Setup and start the simulation
print('Fuel Station refuelling')
random.seed(RANDOM_SEED)

# Create environment and start processes
env = simpy.Environment()
fuel_station = simpy.Resource(env, 2)
fuel_pump = simpy.Container(env, FUEL_STATION_SIZE, init=FUEL_STATION_SIZE)
env.process(fuel_station_control(env, fuel_pump))
env.process(car_generator(env, fuel_station, fuel_pump))

# Execute!
env.run(until=SIM_TIME)

Fuel Station refuelling
Car 0 arriving at fuel station at 87.0
Car 0 finished refueling in 18.5 seconds.
Car 1 arriving at fuel station at 129.0
Car 1 finished refueling in 19.0 seconds.
Car 2 arriving at fuel station at 284.0
Car 2 finished refueling in 21.0 seconds.
Car 3 arriving at fuel station at 385.0
Car 3 finished refueling in 13.5 seconds.
Car 4 arriving at fuel station at 459.0
Calling tank truck at 460
Car 4 finished refueling in 22.0 seconds.
Car 5 arriving at fuel station at 705.0
Car 6 arriving at fuel station at 750.0
Tank truck arriving at time 760
Tank truck refuelling 188.0 liters.
Car 6 finished refueling in 29.0 seconds.
Car 5 finished refueling in 76.5 seconds.
Car 7 arriving at fuel station at 891.0
Car 7 finished refueling in 13.0 seconds.
